<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/%E5%A4%9A%E9%A0%85%E9%81%B8%E6%93%87/demo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [2]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForMultipleChoice
import numpy as np

# Load the dataset
datasets = load_dataset('roberthsu2003/for_Multiple_Choice')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

# Process function
def process_function(examples):
    contexts = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = examples['context'][idx]
        question = examples['question'][idx]

        choices = examples["choices"][idx]
        for choice in choices:
            contexts.append(ctx)
            question_choice.append(question + " " + choice)

        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                contexts.append(ctx)
                question_choice.append(question + " " + "不知道")
        answer = examples['answer'][idx]
        labels.append(choices.index(answer))
    tokenizer_example = tokenizer(contexts, question_choice, truncation="only_first", max_length=256, padding="max_length")
    tokenized_example = {k: [v[i:i+4] for i in range(0,len(v),4)] for k, v in tokenizer_example.items()}
    tokenized_example['labels'] = labels #add labels
    return tokenized_example

# Tokenize the dataset
tokenized_c3 = datasets.map(process_function, batched=True, remove_columns=datasets['train'].column_names)

# Load the model
model = AutoModelForMultipleChoice.from_pretrained('google-bert/bert-base-chinese')

# Evaluation metric
accuracy = evaluate.load("accuracy")

def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training arguments
args = TrainingArguments(
    output_dir="./multiple_choice",
    per_device_train_batch_size=16,  # Because it's 3D, 16*4=64, the actual batch size will be 64
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    report_to='none'
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_c3['train'],
    eval_dataset=tokenized_c3['validation'],
    compute_metrics=compute_metric
)

# Train the model
#trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/504 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/364k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5856 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/5856 [00:00<?, ? examples/s]

Map:   0%|          | 0/1825 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 